In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score, roc_curve, auc, classification_report
from datasets import Dataset
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
# -----------------------
# Load and Prepare Dataset
# -----------------------
df = pd.read_csv("train.csv")

In [ ]:
df.info()

In [ ]:
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
df = df.dropna(subset=['comment_text'])
df[label_cols] = df[label_cols].astype(int)

In [ ]:
df.info()

In [ ]:
# Train-test split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [ ]:
# -----------------------
# Tokenization
# -----------------------
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize(batch):
    return tokenizer(batch["comment_text"], padding="max_length", truncation=True, max_length=128)

train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

In [ ]:
# Convert labels to float for BCE loss
train_dataset = train_dataset.map(lambda x: {"labels": [float(x[col]) for col in label_cols]})
test_dataset = test_dataset.map(lambda x: {"labels": [float(x[col]) for col in label_cols]})

train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
test_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

In [ ]:
# -----------------------
# Model
# -----------------------
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=len(label_cols),
    problem_type="multi_label_classification"
)

In [ ]:
# -----------------------
# Metrics
# -----------------------
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = 1 / (1 + np.exp(-logits))
    preds = (probs >= 0.5).astype(int)

    f1 = f1_score(labels, preds, average="macro")
    roc_auc = roc_auc_score(labels, probs, average="macro")
    acc = accuracy_score(labels, preds)
    return {"f1": f1, "roc_auc": roc_auc, "accuracy": acc}


In [ ]:
# -----------------------
# Training Arguments
# -----------------------

training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy="epoch",        # Evaluate at the end of each epoch
    save_strategy="epoch",              # Save checkpoint each epoch
    learning_rate=3e-5,                 # Reasonable learning rate for DistilBERT
    per_device_train_batch_size=16,     # Smaller batch size for faster iteration
    per_device_eval_batch_size=16,
    num_train_epochs=2,                 # Reduce epochs for quick training
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=50,                   # Log less frequently
    load_best_model_at_end=True,
    metric_for_best_model="f1",         # Use F1 to track best model
    greater_is_better=True,
    fp16=True,                          # Use mixed precision (if GPU supports it)
    report_to="none",                   # Disable W&B or other tracking
)


In [ ]:
# -----------------------
# Trainer
# -----------------------
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

In [ ]:
# -----------------------
# Evaluation and Predictions
# -----------------------
results = trainer.evaluate()
print("\n📊 Evaluation Results:", results)

# Predict probabilities and binary labels
predictions = trainer.predict(test_dataset)
logits = predictions.predictions
probs = 1 / (1 + np.exp(-logits))
binary_preds = (probs >= 0.5).astype(int)

In [ ]:
# -----------------------
# Plot ROC Curves
# -----------------------
for i, label in enumerate(label_cols):
    fpr, tpr, _ = roc_curve(predictions.label_ids[:, i], probs[:, i])
    plt.plot(fpr, tpr, label=f"{label} (AUC={auc(fpr, tpr):.3f})")

plt.plot([0, 1], [0, 1], "k--")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curves - Multi-Label Toxic Comment Classification")
plt.legend()
plt.show()


In [ ]:
from sklearn.metrics import classification_report, roc_auc_score

# Get true and predicted labels
y_true = predictions.label_ids
y_probs = probs
y_pred = binary_preds

label_names = label_cols
# 🔹 Detailed classification report (Precision, Recall, F1)
report = classification_report(y_true, y_pred, target_names=label_names, output_dict=True)
print("Classification Report (Per Label):")
for label in label_names:
    print(f"{label:15s} | Precision: {report[label]['precision']:.3f} | "
          f"Recall: {report[label]['recall']:.3f} | "
          f"F1-score: {report[label]['f1-score']:.3f}")

In [ ]:
# 🔹 AUROC for each label
print("\nAUROC Scores (Per Label):")
for i, label in enumerate(label_names):
    auc_score = roc_auc_score(y_true[:, i], y_probs[:, i])
    print(f"{label:15s} | AUC: {auc_score:.3f}")

In [ ]:
# -----------------------
# 🔮 Real-world Prediction
# -----------------------
def predict_text(text):
    # Tokenize input
    inputs = tokenizer(
        text,
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors="pt"
    ).to(model.device)

    # Get model outputs
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = torch.sigmoid(logits).cpu().numpy()[0]

    # Binary predictions (threshold 0.5)
    preds = (probs >= 0.5).astype(int)

    # Prepare readable output
    results = {label_cols[i]: float(probs[i]) for i in range(len(label_cols))}
    binary_results = {label_cols[i]: int(preds[i]) for i in range(len(label_cols))}

    return results, binary_results


In [ ]:
# Example usage
sample_text = "You are such an idiot, stop posting nonsense!"
predict_text(sample_text)

In [ ]:
from tqdm import tqdm

# Load your test dataset
test_df = pd.read_csv("test.csv")

# Prepare lists to store results
all_probs = []
all_preds = []

# Set model to evaluation mode
model.eval()

# Disable gradient calculation for faster inference
with torch.no_grad():
    for text in tqdm(test_df["comment_text"], desc="Predicting"):
        # Tokenize
        inputs = tokenizer(
            text,
            padding=True,
            truncation=True,
            max_length=128,
            return_tensors="pt"
        ).to(model.device)

        # Forward pass
        outputs = model(**inputs)
        logits = outputs.logits
        probs = torch.sigmoid(logits).cpu().numpy()[0]
        preds = (probs >= 0.5).astype(int)

        all_probs.append(probs)
        all_preds.append(preds)

# Convert to DataFrames
prob_df = pd.DataFrame(all_probs, columns=[f"{col}_prob" for col in label_cols])
pred_df = pd.DataFrame(all_preds, columns=label_cols)

# Combine with IDs and comments
final_df = pd.concat([test_df[["id", "comment_text"]].reset_index(drop=True), pred_df], axis=1)


In [ ]:
final_df

In [ ]:
final_df.to_csv('test_result.csv', index=False)

In [ ]:
# -----------------------
# Save Model
# -----------------------
# Save the model
trainer.save_model("./toxic_comment_model")

print("Model saved successfully to ./toxic_comment_model")

In [ ]:
import shutil
import os

# Define the directory to be zipped and the name of the zip file
directory_to_zip = "./toxic_comment_model"
zip_file_name = "toxic_comment_model.zip"

# Create the zip file
shutil.make_archive(zip_file_name.replace('.zip', ''), 'zip', directory_to_zip)

print(f"'{directory_to_zip}' has been zipped to '{zip_file_name}'. You can now download this file from the file browser.")